In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

# Introduction

For organisations of 250 or more employees in the UK, the gender pay gap has been reported as 6.5% (mean) and 15.9% (median).

https://www.gov.uk/government/publications/dit-gender-pay-gap-report-and-data-2019-to-2020/dit-gender-pay-gap-report-2019-to-2020

There are many different arguments for why this gap exists. One of the more prominent arguments is that work that has a woman majority workforce is less valued by society than work of similar value that has a man majority.

https://www.epi.org/publication/womens-work-and-the-gender-pay-gap-how-discrimination-societal-norms-and-other-forces-affect-womens-occupational-choices-and-their-pay/

https://www.striking-women.org/module/workplace-issues-past-and-present/gender-pay-gap-and-struggle-equal-pay

I hope to shed some light on this from a fresh angle.

# Data

The data used is the OECD Survey of Adult Skills (PIAAC), a survey that took place globally from 2011-2017.

https://www.oecd.org/skills/piaac/data/

The survey had over 1500 questions in total and about 3600 responses that were public use. Of these 3600, I am only looking at non self-employed respondents where a monthly salary was available. The final list of responses comes to about 2000, which isn't a great number but the quality of the responses is great. I reduced the many many questions to just ~30 that cover the sex of the respondent and many aspects about they're job including salary.

For a full list of column explainations see the data description:

https://www.kaggle.com/hekkta/salary-predictor

All data preparation has been done before uploading the dataset.

In [ ]:
df = pd.read_csv('/kaggle/input/salary-predictor/wage_predict.csv')

print('The data set has ' + str(df.shape[0]) + ' rows and ' + str(df.shape[1]) + ' columns.')

df.head()

# Analysis

## How do can we detect a bias?

As a society we give a value to every job, this is represented by it's salary (mostly). To see whether there is a bias in overvaluing male majority jobs, we'll first find out how we measure the value of a non-majority job i.e what makes a job well payed and what makes it poorly payed. We use 30 different aspects of a job that have been hand picked to offer the best results. Then we can predict the values of male and female majority jobs and compare them to their actual values to see if we are under/overvaluing them.


## Which features do we value in non-majority jobs?

Before diving into building a model it would be nice to have a feel for what elements of a job have the most say in determining it's salary. The variance of salaries within the features is a simple and clear way to do this.

### Variance of salary

The 'computer' feature is a binary choice of:

1 - 'yes the job uses a computer'

2 - 'no the job does not use a computer'

If the average 'yes' response has the same monthly salary as the average 'no' response, then this feature is a terrible indicator of salary. This is equivalent to saying that the variance of the average salaries of the responses is 0.

In [ ]:
computer = df[['computer', 'monthly_salary']].groupby(by='computer').mean()
computer = computer.rename(columns={'monthly_salary':'average monthly salary'})
computer

Jobs that need a computer on average have a salary that doubles the jobs that don't need a computer. This has a high variance and is a great indicator of salary. Let's compare this with the 'sector' feature. The sector feature has 3 categories:

1 - The private sector

2 - The public sector

3 - A non-profit organisation

In [ ]:
df[['sector', 'monthly_salary']].groupby(by='sector').mean()

Working in the private sector is certainly less profitable, but as a feature 'sector' is not a good indicator of salary. Let's take a look and compare all features.

In [ ]:
importance = pd.DataFrame([], columns=df.columns[:-1])

# find the variances for each feature
for column in df.columns[:-1]:
    importance.loc[0, column] = df[[column, 'monthly_salary']].groupby(by=column).mean().var(axis=0)[0]
    
# plot the variance
fig, ax = plt.subplots()

fig.set_size_inches(11.7, 8.27)
    
var = sns.barplot(ax= ax, data=importance, orient = 'h').set_title('Variance of salary within the features', fontsize=17)

ax.set_xlabel('Variance', fontsize=14);

## Model building

To build our model we only care about non-majority jobs. Let's take a look at sex distribution for each job industry.

In [ ]:
df_gender = df[['industry', 'sex']]

industry_key = {'A':'Agriculture, forestry and fishing','B':'Mining and quarrying', 'C':'Manufacturing', 'D':'Electricity, gas, steam and air conditioning supply',
                'E':'Water supply; sewerage, waste management and remediation activities', 'F':'Construction', 'G':'Wholesale and retail trade; repair of motor vehicles',
                'H':'Transportation and storage', 'I':'Accommodation and food service activities', 'J':'Information and communication', 'K':'Financial and insurance activities',
                'L':'Real estate activities', 'M':'Professional, scientific and technical activities', 'N':'Administrative and support service activities',
                'O':'Public administration and defence; compulsory social security','P':'Education','Q':'Human health and social work activities',
                'R':'Arts, entertainment and recreation','S':'Other service activities','T':'Activities of households as employers',
                'U':'Activities of extraterritorial organizations and bodies'}

industry_list = df['industry'].unique()
male_list = []
male_count = []
female_count = []

for industry in industry_list:
    count = df_gender[df_gender['industry'] == industry].value_counts()/ df_gender[df_gender['industry'] == industry].shape[0]
    count2 = df_gender[df_gender['industry'] == industry].value_counts()
    try:
        male = count.loc[:, 1]
        male2 = count2.loc[:, 1]
    except:
        male = [0]
        male2 = [0]
    try:
        female2 = count2.loc[:, 2]
    except:
        female2 = [0]
        
    male_list.append(male[0]*100)
    male_count.append(male2[0])
    female_count.append(female2[0])
    
df2 = pd.DataFrame(industry_list, columns = ['Industry Code'])

df2['Industry'] = df2['Industry Code'].apply(lambda x: industry_key[x])

df2['Male (%)'] = male_list

df2['Female (%)'] = 100 - df2['Male (%)']

df2['Male count'] = male_count

df2['Female count'] = female_count


df2.sort_values(by='Male (%)')

60% majority is a good cut-off to divide the industries whilst keeping a large sample size in each partition. Thus the non-majority industry list looks like this.

In [ ]:
non_industry_list = list(df2[(df2['Male (%)'] < 60) & (df2['Male (%)'] > 40)]['Industry Code'])
male_industry_list = list(df2[(df2['Male (%)'] >= 60)]['Industry Code'])
female_industry_list = list(df2[(df2['Male (%)'] <= 40)]['Industry Code'])

df_non = df2[(df2['Male (%)'] < 60) & (df2['Male (%)'] > 40)]
df_non

Building the model is just a matter of one-hot encoding the categorical features and then fitting a regression line.

In [ ]:
# dont consider 'industry' or'salary' and don't consider non-job specific features 'total years schooling' or 'sex'
X = df.iloc[:, 1:-1].drop(columns=['sex', 'yrs_qual'])

# only consider 'salary'
y = pd.DataFrame(df.iloc[:, -1])

# define the onehot encoder
def encode(columns, dataframe):
    for column in columns:
        for category in dataframe[column].unique():
            dataframe[column + '_' + str(category)] = 0
            dataframe[column + '_' + str(category)][dataframe[column] == category] = 1
        dataframe = dataframe.drop(columns=column)
    return dataframe

# encode features
X = encode(['occupation', 'area_of_study', 'sector','workforce_change', 'qual_needed'], X)

# split by industry
X['industry'] = df['industry']
X_non = X[X['industry'].isin(non_industry_list)].drop(columns='industry')
X_male = X[X['industry'].isin(male_industry_list)].drop(columns='industry')
X_female = X[X['industry'].isin(female_industry_list)].drop(columns='industry')

y['industry'] = df['industry']
y_non = y[y['industry'].isin(non_industry_list)].drop(columns='industry')
y_male = y[y['industry'].isin(male_industry_list)].drop(columns='industry')
y_female = y[y['industry'].isin(female_industry_list)].drop(columns='industry')

X_train, X_test, y_train, y_test = train_test_split(X_non, y_non, test_size=0.20, random_state=100)

reg = LinearRegression().fit(X_train, y_train)#


reg.score(X_test, y_test)

Given the small size of the data set, the random_state of our train test split has a great impact on model performance. This must be considered when interpreting our results. Predicting the monthly salaries of male and female majority industries with our gender neutral model.

In [ ]:
pred_male = reg.predict(X_male)
pred_female = reg.predict(X_female)

diff_male = (pred_male - y_male).mean()
diff_female = (pred_female - y_female).mean()

df_diff = pd.DataFrame([])

df_diff['Male dominated actual'] = y_male.mean()
df_diff['Male dominated predicted'] = pred_male.mean()
df_diff['Female dominated actual'] = y_female.mean()
df_diff['Female dominated predicted'] = pred_female.mean()


# plot the variance
fig, ax = plt.subplots()

fig.set_size_inches(11.7, 8.27)

var = sns.barplot(ax= ax, data=df_diff, palette=sns.color_palette("Paired", 4))
sns.set_theme(style="whitegrid")

def change_width(ax, new_value) :
    count = 0
    for patch in ax.patches :
        current_width = patch.get_width()
        diff = current_width - new_value

        # we change the bar width
        patch.set_width(new_value)
        
        if count%2 == 0:
            # we recenter the bar
            patch.set_x(patch.get_x() + diff * 1)
        
        count += 1

change_width(ax, .45)

ax.set_ylabel('Average monthly salary ($)', fontsize=16)
ax.set_title('Average monthly salary ($) of industry type', fontsize=17);

## What did we find?

1. Male dominated industries had salaries that ranged from \\$450 to \\$550 higher per month than predicted. Whereas female dominated industries had salaries that ranged from \\$100 lower to \\$100 higher than predicted. 

2. Our model predicted that female dominated industries should be paying from \\$0 to \\$250 more than the male dominated industries.

The ranges given are due to changing of the random seed when splitting our data for training and testing. The wide ranges show a certain amount of error inherent in our small sample size.

These findings should of course be taken with a pinch of salt. There are many factors that are not considered that could have an impact on our outcome. Here's a few of them.

### Factors that could be affecting the results

1. It's a small dataset, 2000 is such a small sample size that the error will be significant.

2. We can't account for every property that affects salary i.e supply and demand.

3. There are innate biases that can't be accounted for i.e men or women may lie at different rates about how much they are paid.

4. The R^2 score of our linear regression model was ~0.1-0.5 which will affect large/small salaries differently.

There are probably plenty more reasons to assume our results aren't precise. But even with massive margins of error, there would still be some sign of a bias towards male majority jobs in terms of salary. 

## Improvements and follow ups

If I were to follow on this report I would investigate further properties of jobs that affect their salaries like supply and demand as mention above. This would be the likely cause for such skewed results if not bias. This report could be improved with more data. I used the public use files offered by OECD, but would e much better off with the full survey results. Also perhaps a different choice of model would be more advantageous, but this is hard to predict.